# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
customer_products = data.groupby(['CustomerID', 'ProductName']).agg({'Quantity':'sum'}).reset_index()
customer_products

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1
5,33,"Beef - Chuck, Boneless",1
6,33,Beef - Prime Rib Aaa,1
7,33,Beer - Original Organic Lager,1
8,33,Beer - Rickards Red,1
9,33,Black Currants,1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [14]:
prod_cust_pivot = customer_products.pivot_table(values='Quantity', columns='CustomerID', index='ProductName',aggfunc='sum').fillna(0)
prod_cust_pivot.head()
prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = prod_cust_pivot.T
cust_prod_pivot.head()

prod_cust_pivot = prod_cust_pivot.div(prod_cust_pivot.sum(axis=1), axis=0)
cust_prod_pivot = cust_prod_pivot.div(cust_prod_pivot.sum(axis=1), axis=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [15]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'euclidean'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.853471,0.858742,0.853306,0.856845,0.851419,0.848198,0.847844,0.851494,0.847477,...,0.853547,0.846215,0.849391,0.840335,0.851613,0.846271,0.851671,0.846575,0.855182,0.850362
200,0.853471,1.000000,0.857755,0.856816,0.856528,0.852537,0.862793,0.850338,0.857746,0.856018,...,0.864431,0.856246,0.862707,0.853879,0.856899,0.854896,0.869280,0.858026,0.861805,0.856897
264,0.858742,0.857755,1.000000,0.852499,0.860711,0.850284,0.854831,0.846398,0.855225,0.847416,...,0.861643,0.850261,0.853136,0.848543,0.858594,0.860244,0.859258,0.853900,0.865122,0.852414
356,0.853306,0.856816,0.852499,1.000000,0.853916,0.850489,0.853008,0.851379,0.848804,0.845225,...,0.857006,0.848272,0.852503,0.847467,0.851569,0.851808,0.850509,0.854184,0.860359,0.854677
412,0.856845,0.856528,0.860711,0.853916,1.000000,0.854357,0.857896,0.857019,0.856104,0.854800,...,0.858121,0.862552,0.855099,0.849638,0.857780,0.856193,0.854690,0.855880,0.865306,0.861828


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [16]:
similar_cust = list(cust_dist[33].sort_values(ascending=False)[1:].head().index)
similar_cust

[60862, 27672, 33759, 6001, 79458]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]


,ProductName,Quantity
0,Wine - Redchard Merritt,59
1,Cassis,58
2,Bread - Calabrese Baguette,45
3,Wine - Crozes Hermitage E.,45
4,"Thyme - Lemon, Fresh",42


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [ ]:
grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
ranked_products.head()

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [18]:
merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[33]), on='ProductName')
merged.columns = ['ProductName', 'Quantity', 'Purchased']
recs = merged[merged['Purchased']==0].head()
recs

,ProductName,Quantity,Purchased
0,Wine - Redchard Merritt,59,0.0
2,Bread - Calabrese Baguette,45,0.0
4,"Thyme - Lemon, Fresh",42,0.0
9,Milk Powder,39,0.0
10,Ecolab - Lime - A - Way 4/4 L,39,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [19]:
recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [20]:
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Wine - Redchard Merritt,Bread - Calabrese Baguette,"Thyme - Lemon, Fresh",Milk Powder,Ecolab - Lime - A - Way 4/4 L
1,200,Sauce - Demi Glace,Pasta - Angel Hair,Tomatoes Tear Drop,Grouper - Fresh,Vaccum Bag 10x13
2,264,Scallops - 10/20,Wine - Ej Gallo Sierra Valley,Cheese - Camembert,Corn Meal,Browning Caramel Glace
3,356,Squid U5 - Thailand,Clam Nectar,"Turnip - White, Organic","Yogurt - Blueberry, 175 Gr",Garbag Bags - Black
4,412,Beef - Montreal Smoked Brisket,Scallops - 10/20,Onions - Cippolini,Juice - Orange,Cake - Box Window 10x10x2.5


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [21]:
cust_dist = pd.DataFrame(1/(1 + squareform(pdist(cust_prod_pivot, 'jaccard'))),
                         index=cust_prod_pivot.index, columns=cust_prod_pivot.index)

cust_dist.head()

recommendations = {}
customers = list(customer_products['CustomerID'].unique())

for customer in customers:
    similar_cust = list(cust_dist[customer].sort_values(ascending=False)[1:].head().index)
    sim_cust_prod = customer_products[customer_products['CustomerID'].isin(similar_cust)]
    grouped = sim_cust_prod.groupby('ProductName').agg({'Quantity':'sum'})
    ranked_products = grouped.sort_values('Quantity', ascending=False).reset_index()
    
    merged = pd.merge(ranked_products, pd.DataFrame(cust_prod_pivot.T[customer]), on='ProductName')
    merged.columns = ['ProductName', 'Quantity', 'Purchased']
    recs = merged[merged['Purchased']==0].head()
    recommendations[customer] = list(recs['ProductName'])
    
user_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
user_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
user_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33,Jolt Cola - Electric Blue,Durian Fruit,Snapple - Iced Tea Peach,Apricots - Halves,"Water - Mineral, Natural"
1,200,Sea Bass - Whole,Vol Au Vents,"Soup - Campbells, Lentil","Rosemary - Primerba, Paste",Ocean Spray - Kiwi Strawberry
2,264,Sea Bass - Whole,Vol Au Vents,"Soup - Campbells, Lentil","Rosemary - Primerba, Paste",Ocean Spray - Kiwi Strawberry
3,356,Sea Bass - Whole,"Soup - Campbells, Lentil","Rosemary - Primerba, Paste",Ocean Spray - Kiwi Strawberry,Dried Figs
4,412,Sea Bass - Whole,Vol Au Vents,"Soup - Campbells, Lentil","Rosemary - Primerba, Paste",Ocean Spray - Kiwi Strawberry
